<a href="https://colab.research.google.com/github/promckkon/MK-DCNN/blob/main/MK-DCNN%20with%200dB%20NOISE%20in%20TRIAX%20Dataset%20266129.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [250]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [251]:
import scipy.io
import seaborn as sns
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt

Dataset Input

In [252]:
df=pd.DataFrame(columns=['DE_data','fault']) # upload the dataset on googledrive

for root, dirs, files in os.walk("/content/drive/MyDrive/TRIAX_with_NOISE/TRIAX_Dataset", topdown=False):
    for file_name in files:
        path = os.path.join(root, file_name)
        print(path)

        mat = scipy.io.loadmat(path)

        key_name = list(mat.keys())[3]
        DE_data = mat.get(key_name)
        fault = np.full((len(DE_data), 1), file_name[:-4])

        df_temp = pd.DataFrame({'DE_data':np.ravel(DE_data) , 'fault':np.ravel(fault)})

        df = pd.concat([df,df_temp],axis=0)
        print(df['fault'].unique())

df.to_csv('/content/drive/MyDrive/MK DCNN TRIAX/NOISE_0_faults.csv',index=False)

/content/drive/MyDrive/TRIAX_with_NOISE/TRIAX_Dataset/F8_Normal_100W.mat
['F8_Normal_100W']
/content/drive/MyDrive/TRIAX_with_NOISE/TRIAX_Dataset/F1_IR007_100W.mat
['F8_Normal_100W' 'F1_IR007_100W']
/content/drive/MyDrive/TRIAX_with_NOISE/TRIAX_Dataset/F2_IR009_100W.mat
['F8_Normal_100W' 'F1_IR007_100W' 'F2_IR009_100W']
/content/drive/MyDrive/TRIAX_with_NOISE/TRIAX_Dataset/F3_IR013_100W.mat


/tmp/ipython-input-4228204961.py:16: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df,df_temp],axis=0)


['F8_Normal_100W' 'F1_IR007_100W' 'F2_IR009_100W' 'F3_IR013_100W']
/content/drive/MyDrive/TRIAX_with_NOISE/TRIAX_Dataset/F4_IR017_100W.mat
['F8_Normal_100W' 'F1_IR007_100W' 'F2_IR009_100W' 'F3_IR013_100W'
 'F4_IR017_100W']
/content/drive/MyDrive/TRIAX_with_NOISE/TRIAX_Dataset/F5_OR009_100W.mat
['F8_Normal_100W' 'F1_IR007_100W' 'F2_IR009_100W' 'F3_IR013_100W'
 'F4_IR017_100W' 'F5_OR009_100W']
/content/drive/MyDrive/TRIAX_with_NOISE/TRIAX_Dataset/F6_OR013_100W.mat
['F8_Normal_100W' 'F1_IR007_100W' 'F2_IR009_100W' 'F3_IR013_100W'
 'F4_IR017_100W' 'F5_OR009_100W' 'F6_OR013_100W']
/content/drive/MyDrive/TRIAX_with_NOISE/TRIAX_Dataset/F7_OR017_100W.mat
['F8_Normal_100W' 'F1_IR007_100W' 'F2_IR009_100W' 'F3_IR013_100W'
 'F4_IR017_100W' 'F5_OR009_100W' 'F6_OR013_100W' 'F7_OR017_100W']


# Multi-Kernel Deep 1D-CNN

In [253]:
pip install tensorflow

In [254]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical


# Total target rows
TARGET_ROWS = 1800

# Estimate the total number of samples across all fault types
total_samples = sum(len(df[df['fault'] == f]) for f in df['fault'].unique())

# Calculate suitable window size and stride
num_faults = 9
average_samples_per_fault = total_samples / num_faults
approx_windows_per_fault = TARGET_ROWS / num_faults
stride_ratio = 0.8  # Initial stride/window ratio

win_len = int(average_samples_per_fault / approx_windows_per_fault)
stride = int(win_len * stride_ratio)

X=[]
Y=[]


for k in df['fault'].unique():

    df_temp_2 = df[df['fault']==k]

    for i in np.arange(0,len(df_temp_2)-(win_len),stride):
        temp = df_temp_2.iloc[i:i+win_len,:-1].values
        temp = temp.reshape((1,-1))
        X.append(temp)
        Y.append(df_temp_2.iloc[i+win_len,-1])

X=np.array(X)
X=X.reshape((X.shape[0],-1,1))
#X = np.repeat(X, 3, axis=3) # To repeat into 3 chanel format


Y=np.array(Y)
encoder= LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
OHE_Y = to_categorical(encoded_Y)

In [255]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,OHE_Y,test_size=0.3,shuffle=True)

In [256]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, Flatten, Conv1D, MaxPooling1D, concatenate
from tensorflow.keras import backend as K

# --- Custom Physics-Informed Loss Function ---
def custom_loss(y_true, y_pred):
    # Standard categorical crossentropy
    loss = K.categorical_crossentropy(y_true, y_pred)

    # Physics-Informed Term: penalize rapid class probability changes
    # Ensure y_pred is at least 2D for slicing
    if K.ndim(y_pred) < 2:
        y_pred = K.expand_dims(y_pred, axis=-1)

    # Ensure y_pred has more than one class dimension to compute diff
    if K.int_shape(y_pred)[-1] > 1:
        diff = y_pred[:, 1:] - y_pred[:, :-1]
        squared_diff = tf.square(diff)
        physics_term = tf.reduce_mean(squared_diff)
    else:
        physics_term = 0.0 # No physics term if only one class


    # Total loss = classification loss + regularization term
    total_loss = loss + 0.01 * physics_term  # 0.01 is tunable

    return total_loss

# --- Model Definition ---
# no_classes = len(df['fault'].unique()) # Original line that caused 10 classes
no_classes = len(encoder.classes_) # Use the encoder to get the correct number of classes
print(f"Number of output classes (no_classes): {no_classes}") # Add print statement to verify

input_shape = (X.shape[1], X.shape[2])  # Example: (784, 1)

# Head 1
inputs1 = Input(shape=input_shape)
conv1 = Conv1D(filters=64, kernel_size=200, activation='relu')(inputs1)
conv11 =(Conv1D(filters=32, kernel_size=50, activation='relu'))(conv1)
drop1 = Dropout(0.5)(conv11)
flat1 = Flatten()(drop1)

# Head 2
inputs2 = Input(shape=input_shape)
conv2 = Conv1D(filters=64, kernel_size=100, activation='relu')(inputs2)
conv21 =(Conv1D(filters=32, kernel_size=50, activation='relu'))(conv2)
drop2 = Dropout(0.5)(conv21)
flat2 = Flatten()(drop2)

# Head 3
inputs3 = Input(shape=input_shape)
conv3 = Conv1D(filters=64, kernel_size=50, activation='relu')(inputs3)
conv31 =(Conv1D(filters=32, kernel_size=50, activation='relu'))(conv3)
drop3 = Dropout(0.5)(conv31)
flat3 = Flatten()(drop3)

# Merge all feature paths
merged = concatenate([flat1, flat2, flat3])

# Fully connected interpretation
dense1 = Dense(50, activation='relu')(merged)
outputs = Dense(no_classes, activation='softmax')(dense1)

# Build and compile model with custom loss
cnn_model = Model(inputs=[inputs1, inputs2, inputs3], outputs=outputs)
cnn_model.compile(optimizer='adam', loss=custom_loss, metrics=['accuracy'])

# Model summary
cnn_model.summary()

Number of output classes (no_classes): 8


Model: "functional_263"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_57      │ (None, 614, 1)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_58      │ (None, 614, 1)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_59      │ (None, 614, 1)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_108 (Conv1D) │ (None, 415, 64)   │     12,864 │ input_layer_57[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_110 (Conv1D) │ (None, 515, 64)   │      6,464 │ input_layer_58[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_112 (Conv1D) │ (None, 565, 64)   │      3,264 │ input_layer_59[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_109 (Conv1D) │ (None, 366, 32)   │    102,432 │ conv1d_108[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_111 (Conv1D) │ (None, 466, 32)   │    102,432 │ conv1d_110[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_113 (Conv1D) │ (None, 516, 32)   │    102,432 │ conv1d_112[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_68          │ (None, 366, 32)   │          0 │ conv1d_109[0][0]  │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_69          │ (None, 466, 32)   │          0 │ conv1d_111[0][0]  │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_70          │ (None, 516, 32)   │          0 │ conv1d_113[0][0]  │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_57          │ (None, 11712)     │          0 │ dropout_68[0][0]  │
│ (Flatten)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_58          │ (None, 14912)     │          0 │ dropout_69[0][0]  │
│ (Flatten)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_59          │ (None, 16512)     │          0 │ dropout_70[0][0]  │
│ (Flatten)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_5       │ (None, 43136)     │          0 │ flatten_57[0][0], │
│ (Concatenate)       │                   │            │ flatten_58[0][0], │
│                     │                   │            │ flatten_59[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_94 (Dense)    │ (None, 50)        │  2,156,850 │ concatenate_5[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_95 (Dense)    │ (None, 8)         │        408 │ dense_94[0][0]    │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 2,487,146 (9.49 MB)

 Trainable params: 2,487,146 (9.49 MB)

 Non-trainable params: 0 (0.00 B)

In [257]:
batch_size =200
epochs = 60
history = cnn_model.fit([X_train,X_train,X_train], y_train, batch_size=batch_size,epochs=epochs,verbose=1,validation_data=([X_test,X_test,X_test],y_test),shuffle=True)

Epoch 1/60
8/8 ━━━━━━━━━━━━━━━━━━━━ 23s 943ms/step - accuracy: 0.1301 - loss: 2.2106 - val_accuracy: 0.1306 - val_loss: 2.0361
Epoch 2/60
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 79ms/step - accuracy: 0.1366 - loss: 1.9776 - val_accuracy: 0.2255 - val_loss: 1.8288
Epoch 3/60
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 67ms/step - accuracy: 0.2700 - loss: 1.7781 - val_accuracy: 0.3145 - val_loss: 1.5162
Epoch 4/60
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 67ms/step - accuracy: 0.3398 - loss: 1.4658 - val_accuracy: 0.4036 - val_loss: 1.3079
Epoch 5/60
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 71ms/step - accuracy: 0.4212 - loss: 1.2580 - val_accuracy: 0.5519 - val_loss: 1.1678
Epoch 6/60
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 63ms/step - accuracy: 0.6425 - loss: 1.1198 - val_accuracy: 0.6573 - val_loss: 1.0136
Epoch 7/60
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - accuracy: 0.6815 - loss: 0.9638 - val_accuracy: 0.6721 - val_loss: 0.9779
Epoch 8/60
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 62ms/step - accuracy: 0.7418 - loss: 0.8883 - val_accuracy: 0.7240 - val_loss: 0.883

In [261]:
import numpy as np
from sklearn.metrics import classification_report

def inv_transform_result(y):
    y = y.argmax(axis=1)
    return encoder.inverse_transform(y)

# ===== 預測 =====
y_train_pred = cnn_model.predict([X_train, X_train, X_train])
y_test_pred  = cnn_model.predict([X_test, X_test, X_test])

Y_train_pred = inv_transform_result(y_train_pred)
Y_test_pred  = inv_transform_result(y_test_pred)

Y_train_true = inv_transform_result(y_train)
Y_test_true  = inv_transform_result(y_test)

# ===== Classification Report =====
print("Classification Report - Training Set:\n")
print(classification_report(Y_train_true, Y_train_pred, target_names=encoder.classes_))

print("\nClassification Report - Test Set:\n")
print(classification_report(Y_test_true, Y_test_pred, target_names=encoder.classes_))

50/50 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
Classification Report - Training Set:

                precision    recall  f1-score   support

 F1_IR007_100W       1.00      1.00      1.00       213
 F2_IR009_100W       1.00      1.00      1.00       225
 F3_IR013_100W       1.00      1.00      1.00       194
 F4_IR017_100W       1.00      1.00      1.00       190
 F5_OR009_100W       1.00      1.00      1.00       196
 F6_OR013_100W       1.00      1.00      1.00       186
 F7_OR017_100W       1.00      1.00      1.00       198
F8_Normal_100W       1.00      1.00      1.00       170

      accuracy                           1.00      1572
     macro avg       1.00      1.00      1.00      1572
  weighted avg       1.00      1.00      1.00      1572


Classification Report - Test Set:

                precision    recall  f1-score   support

 F1_IR007_100W       0.99      0.97      0.98        78
 F2_IR009_100W       0.90      0.85      0.87        85
 